In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Notebooks used as sources:

https://www.kaggle.com/uysimty/keras-cnn-dog-or-cat-classification

https://www.kaggle.com/bulentsiyah/dogs-vs-cats-classification-vgg16-fine-tuning

### Libraries Import

In [ ]:
import numpy as np
import pandas as pd 

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, GlobalMaxPooling2D
from keras.applications import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback, LearningRateScheduler
from keras.optimizers import RMSprop
from keras import backend

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import random
import os
import zipfile

### Input Handling

In [ ]:
print(os.listdir("/kaggle/input/dogs-vs-cats/"))

with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r") as z:
    z.extractall("/kaggle/working/dogs-vs-cats/")
    
# with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip","r") as z:
#     z.extractall("/kaggle/working/dogs-vs-cats/")

In [ ]:
filenames = os.listdir("/kaggle/working/dogs-vs-cats/train")

df = pd.DataFrame()

df['filename'] = filenames
df['category'] = df['filename'].apply(lambda x: x.split('.')[0])
df['label'] = np.where(df['category'] == 'cat', 0, 1)
df.head(n=10)

In [ ]:
print(df['category'].value_counts())
df['category'].value_counts().plot.bar()

### Sample Image

In [ ]:
image = load_img("/kaggle/working/dogs-vs-cats/train/" + random.choice(filenames))
plt.imshow(image)

### Prepare Data

In [ ]:
df.head()

In [ ]:
train_df, validate_df = train_test_split(df.drop(columns = 'label'), test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
print(train_df['category'].value_counts())
print(train_df.shape)
print('-'*60)
print(validate_df['category'].value_counts())
print(validate_df.shape)

In [ ]:
train_df

Read images from directory using Keras

https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

https://vijayabhaskar96.medium.com/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=16


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    #rotation_range=15,
    #zoom_range=0.2,
    #height_shift_range=0.2,
    #width_shift_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/kaggle/working/dogs-vs-cats/train/", 
    x_col='filename',
    y_col='category',
    target_size=(128,128),
    class_mode='categorical',
    batch_size=batch_size 
    #images served in each epoch
)

valid_datagen = ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_dataframe(
    validate_df, 
    "/kaggle/working/dogs-vs-cats/train/", 
    x_col='filename',
    y_col='category',
    target_size=(128,128),
    class_mode='categorical',
    batch_size=batch_size 
    #images served in each epoch
)

Plotting example images from generator

https://stackoverflow.com/questions/59217031/plot-images-from-image-generator

In [ ]:
example_df = train_df.sample(n=1).reset_index(drop=True)
example_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=15,
    zoom_range=0.2,
    #height_shift_range=0.2,
    #width_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
).flow_from_dataframe(
    example_df, 
    "/kaggle/working/dogs-vs-cats/train/", 
    x_col='filename',
    y_col='category',
    target_size=(128,128),
    class_mode='categorical',
    batch_size = 1
)

plt.figure(figsize=(12, 12))
for i in range(0, 16):
    plt.subplot(4, 4, i+1)
    x = example_generator.next()
    image = x[0][0]
    plt.imshow(image)
plt.tight_layout()
plt.show()

fit_generator(
    generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None,
    validation_data=None, validation_steps=None, validation_freq=1,
    class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False,
    shuffle=True, initial_epoch=0
)

In [ ]:
!nvidia-smi

### Model Building

Convolutional Neural Networks work by extracting features maps from the image using filters/kernels.
Pooling layers are used for dimensionality reduction.


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dropout(0.25)) #Francois Chollet place drop out after flatten in his book Deep Learning with Python (Ch 5, Page 141)
model.add(Dense(512, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25)) 
model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes

opt = RMSprop(learning_rate=0.01)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()

### Batch Normalization

Batch Normalization layer normalizes its input. 
Batch normalization applies a transformation that maintains the mean output close to 0 and the output standard deviation close to 1. This allows each layer in the network to train more independently. 
During the training stage of networks, as the parameters of the preceding layers change, the distribution of inputs to the current layer changes accordingly, such that the current layer needs to constantly readjust to new distributions. This problem is especially severe for deep networks, because small changes in shallower hidden layers will be amplified as they propagate within the network, resulting in significant shift in deeper hidden layers. Therefore, the method of batch normalization is proposed to reduce these unwanted shifts to speed up training and to produce more reliable models.


Sources:
https://analyticsindiamag.com/everything-you-should-know-about-dropouts-and-batchnormalization-in-cnn/

https://en.wikipedia.org/wiki/Batch_normalization

https://keras.io/api/layers/normalization_layers/batch_normalization/

### Softmax
Takes a vector of length k with real values. Returns a vector K of same length such that K[i] $\in$ [0,1] and np.sum(K) = 1

### Cross Entropy

Loss Function: L(y, y') = - (y*log(y') + (1-y)*log(1-y'))

if y = 1 ==> L(1, y') = -log(y') ==> y' must be as large as possible ==> y' -> 1

if y = 0 ==> L(0, y') = -log(1-y') ==> 1-y' must be as large as possible ==> y' must be as small as possible ==> y' -> 0

The loss function computes the error for a single training example; the cost function is the average of the loss functions
of the entire training set.

Cost Function: J = 1/m * Sum (L(y[i], y'[i])) 

For more classes: Say you have 3 classes:
Loss function = - [y1log(y1') + y2log(y2') + y3log(y3')]
For multiclass classification, due to one-hot encoding, we will have only one term active. So again the idea is to maximize output score for active class so it is closer to 1.

Non-trainable parameters:

https://stackoverflow.com/questions/47312219/what-is-the-definition-of-a-non-trainable-parameter


### Callbacks

In [ ]:
lr_0 = 0.01
earlystop = EarlyStopping(patience=10)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=0.001)

checkpoint_filepath = '/kaggle/working/catsdogsmodelweights.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# def scheduler(epoch, lr):
#     if epoch < 10:
#         return lr
#     else:
#         #return lr * tf.math.exp(-0.1)
#         return lr_0 * 0.6 ** epoch

# lr_decay = tf.keras.callbacks.LearningRateScheduler(scheduler)

# print the learning rate
class LearningRateMonitor(Callback):
#end of each training epoch
    def on_epoch_end(self, epoch, logs=None):
    # print optimizer learning rate
        optimizer = self.model.optimizer
        print("\nLearning rate: {}".format(backend.get_value(model.optimizer.lr)))

lrm = LearningRateMonitor()

callbacks = [earlystop, reduce_lr, lrm, model_checkpoint_callback]

model.fit/fit_generator parameters:

* batch_size
* steps per epoch
* validation_steps

https://datascience.stackexchange.com/questions/29719/how-to-set-batch-size-steps-per-epoch-and-validation-steps

In [ ]:
if not os.path.isfile(checkpoint_filepath):
    print('Pre-trained weights not found. Training from scratch.')
else:
    model.load_weights(checkpoint_filepath)


epochs = 50


In [ ]:

history = model.fit_generator(
    generator = train_generator, 
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=total_validate//batch_size, #floor division
    steps_per_epoch=total_train//batch_size,
    callbacks = callbacks
)
#model.save('/kaggle/working/catsdogsmodel.h5')

In [ ]:
history.history

### Plotting Loss & Accuracy

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="Validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1.1, 0.1))
ax1.set_ylim(0.0,1.0)

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))
ax2.set_yticks(np.arange(0, 1.1, 0.1))
ax2.set_ylim(0.0,1.0)

ax1.legend(loc='best', shadow=True)
ax2.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
# test_filenames = os.listdir("/kaggle/working/dogs-vs-cats/test1")

# test_df = pd.DataFrame()

# test_df['filename'] = test_filenames
# #test_df['category'] = test_df['filename'].apply(lambda x: x.split('.')[0])
# #test_df['label'] = np.where(test_df['category'] == 'cat', 0, 1)
# test_df.head(n=10)

In [ ]:
# test_datagen = ImageDataGenerator(rescale = 1./255)
# test_generator = test_datagen.flow_from_dataframe(
#     test_df, 
#     "/kaggle/working/dogs-vs-cats/test1/", 
#     x_col='filename',
#     y_col=None,
#     target_size=(128,128),
#     class_mode=None,
#     batch_size=15, 
#     #images served in each epoch
#     shuffle = False
# )

https://stackoverflow.com/questions/45806669/keras-how-to-use-predict-generator-with-imagedatagenerator

In [ ]:
y_prob = model.predict_generator(valid_generator)
y_classes = y_prob.argmax(axis=-1)
print(train_generator.class_indices)
print(y_prob[0])
print(y_classes[0])

labels_map = dict((v,k) for (k,v) in train_generator.class_indices.items())
print(labels_map)

validate_df['pred_class'] = y_classes
validate_df['pred_prob'] = y_prob[:,1]
validate_df['pred_label'] = validate_df['pred_class'].replace(labels_map)
validate_df.head(n=10)

### Some Misclassified Examples

In [ ]:
#sample_test = validate_df.head(n=16)
sample_test = validate_df[validate_df['category'] != validate_df['pred_label']].head(n=16).reset_index(drop=True)
plt.figure(figsize=(16, 16))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['pred_label']
    prob = float(row['pred_prob'])
    img = load_img("/kaggle/working/dogs-vs-cats/train/" + filename, target_size=(128,128))
    plt.subplot(4, 4, index+1)
    plt.imshow(img)
    plt.xlabel(filename + ' (' + category + ':' + str(round(prob,2)) + ')' )
plt.tight_layout()
plt.show()

## Using Pre-trained Model: [VGG16](https://keras.io/api/applications/vgg/)

![VGG16](https://miro.medium.com/max/470/1*3-TqqkRQ4rWLOMX-gvkYwA.png)

ImageNet 1000 classes

https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a

In [ ]:
#include_top: whether to include the 3 fully-connected layers at the top of the network
pre_trained_model = VGG16(input_shape=(128,128,3), include_top=False, weights="imagenet")
print("Number of layers:", len(pre_trained_model.layers))
pre_trained_model.summary()

We keep the initial layers as fixed, since they have already been trained and are good at extracting lower level features out of an image.

> Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem. For instance, features from a model that has learned to identify racoons may be useful to kick-start a model meant to identify tanukis.
> Transfer learning is usually done for tasks where your dataset has too little data to train a full-scale model from scratch.
> The most common incarnation of transfer learning in the context of deep learning is the following worfklow:
* Take layers from a previously trained model.
* Freeze them, so as to avoid destroying any of the information they contain during future training rounds.
* Add some new, trainable layers on top of the frozen layers. They will learn to turn the old features into predictions on a new dataset.
* Train the new layers on your dataset.


References:

https://keras.io/guides/transfer_learning/

In [ ]:
# for layer in pre_trained_model.layers[:15]:
#     layer.trainable = False

# for layer in pre_trained_model.layers[15:]:
#     layer.trainable = True

pre_trained_model.trainable = False

Global Max Pooling

https://stats.stackexchange.com/questions/257321/what-is-global-max-pooling-layer-and-what-is-its-advantage-over-maxpooling-layer

Adding layers to the pre-trained model

https://stackoverflow.com/questions/42475381/add-dropout-layers-between-pretrained-dense-layers-in-keras



In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.25
x = Dropout(0.25)(x)
# Add a final sigmoid/softmax layer for classification
x = Dense(2, activation='softmax')(x)

vgg_model = Model(inputs=pre_trained_model.input, outputs=x)

opt = RMSprop(learning_rate=0.01)

vgg_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

vgg_model.summary()

**Callbacks:**

A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc).

https://keras.io/api/callbacks/

In [ ]:
earlystop = EarlyStopping(patience=10)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=0.001)

checkpoint_filepath = '/kaggle/working/vgg16basedcatsdogs.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callbacks = [earlystop, model_checkpoint_callback, reduce_lr]

# if not os.path.isfile(checkpoint_filepath):
#     print('Pre-trained weights not found. Training from scratch.')
# else:
#     vgg_model.load_weights(checkpoint_filepath)


epochs = 50
vgg_history = vgg_model.fit_generator(
    generator = train_generator, 
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=total_validate//batch_size, #floor division
    steps_per_epoch=total_train//batch_size,
    callbacks = callbacks)


In [ ]:
vgg_history.history

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))
ax1.plot(vgg_history.history['loss'], color='b', label="Training loss")
ax1.plot(vgg_history.history['val_loss'], color='r', label="Validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1.1, 0.1))
ax1.set_ylim(0.0,1.0)

ax2.plot(vgg_history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(vgg_history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))
ax2.set_yticks(np.arange(0, 1.1, 0.1))
ax2.set_ylim(0.0,1.0)

ax1.legend(loc='best', shadow=True)
ax2.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
y_prob = vgg_model.predict_generator(valid_generator)
y_classes = y_prob.argmax(axis=-1)
print(train_generator.class_indices)
print(y_prob[0])
print(y_classes[0])

labels_map = dict((v,k) for (k,v) in train_generator.class_indices.items())
print(labels_map)
vgg_df = validate_df[['filename','category']].copy()
vgg_df['pred_class'] = y_classes
vgg_df['pred_prob'] = y_prob[:,1]
vgg_df['pred_label'] = vgg_df['pred_class'].replace(labels_map)
vgg_df.head(n=10)

In [ ]:
sample_test2 = vgg_df[vgg_df['filename'].isin(sample_test['filename'])].reset_index(drop = True)
plt.figure(figsize=(16, 16))
for index, row in sample_test2.iterrows():
    filename = row['filename']
    category = row['pred_label']
    prob = float(row['pred_prob'])
    img = load_img("/kaggle/working/dogs-vs-cats/train/" + filename, target_size=(128,128))
    plt.subplot(4, 4, index+1)
    plt.imshow(img)
    plt.xlabel(filename + ' (' + category + ':' + str(round(prob,2)) + ')' )
plt.tight_layout()
plt.show()

Delete training dataset before saving files

In [ ]:
!rm -r /kaggle/working/dogs-vs-cats